In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
global_path = '/Users/sujie/Desktop/pythonKaggle'

/Users/sujie/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#read the csv file
df_train = pd.read_csv(global_path+'/data/train_users_2.csv')
df_test = pd.read_csv(global_path+'/data/test_users.csv')

In [3]:
np.random.seed(0)

In [4]:
labels = df_train['country_destination'].values

In [5]:
df_train = df_train.drop(['country_destination'], axis=1)

In [6]:
id_test = df_test['id']

In [7]:
piv_train = df_train.shape[0]

In [8]:
#Creating a DataFrame with train+test data
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
#Removing id and date_first_booking
df_all = df_all.drop(['id', 'date_first_booking'], axis=1)
#Filling nan
df_all = df_all.fillna(-1)

In [9]:
dac = np.vstack(df_all.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('/')))).values)
df_all['dac_year'] = dac[:,0]
df_all['dac_month'] = dac[:,1]
df_all['dac_day'] = dac[:,2]
df_all = df_all.drop(['date_account_created'], axis=1)

In [10]:
#timestamp_first_active
tfa = np.vstack(df_all.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
df_all['tfa_year'] = tfa[:,0]
df_all['tfa_month'] = tfa[:,1]
df_all['tfa_day'] = tfa[:,2]
df_all = df_all.drop(['timestamp_first_active'], axis=1)

In [11]:
#Age
av = df_all.age.values
df_all['age'] = np.where(np.logical_or(av<14, av>100), -1, av)

In [12]:
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    df_all_dummy = pd.get_dummies(df_all[f], prefix=f)
    df_all = df_all.drop([f], axis=1)
    df_all = pd.concat((df_all, df_all_dummy), axis=1)


,age,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,gender_-unknown-,gender_FEMALE,gender_MALE,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,-1.0,2010,6,28,2009,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38.0,2011,5,25,2009,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,56.0,2010,9,28,2009,6,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,42.0,2011,12,5,2009,10,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,41.0,2010,9,14,2009,12,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,-1.0,2010,1,1,2010,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,46.0,2010,1,2,2010,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,47.0,2010,1,3,2010,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,50.0,2010,1,4,2010,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,46.0,2010,1,4,2010,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#Splitting train and test
vals = df_all.values
X = vals[:piv_train]
le = LabelEncoder()
y = le.fit_transform(labels)   
X_test = vals[piv_train:]

In [17]:
X_test

array([[  3.50000000e+01,   2.01400000e+03,   7.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -1.00000000e+00,   2.01400000e+03,   7.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -1.00000000e+00,   2.01400000e+03,   7.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [ -1.00000000e+00,   2.01400000e+03,   9.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -1.00000000e+00,   2.01400000e+03,   9.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  4.90000000e+01,   2.01400000e+03,   9.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [14]:
#Classifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(X, y)
y_pred = xgb.predict_proba(X_test)  

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

In [15]:
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])

In [16]:
# sub.to_csv('sub.csv',index=False)

,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,other
3,5uwns89zht,FR
4,5uwns89zht,ES
5,jtl0dijy2j,NDF
6,jtl0dijy2j,US
7,jtl0dijy2j,other
8,jtl0dijy2j,FR
9,jtl0dijy2j,IT
